In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
https = 'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/'
url = requests.get(https).text


soup = BeautifulSoup(url, 'lxml')
#print(soup.prettify())



In [2]:
#we create a list of all useable links, all_links
raw_links = soup.find_all('a')
all_links = []
for link in raw_links:
    if (link.get('href') not in raw_links):
        all_links.append(link.get('href'))

#we create a list of all links to detailed data, csv_links
csv_detailed_links = []        
for link in all_links:
    if((link[-6:-1] == 'csv.g') and link.startswith('StormEvents_details')):
        csv_detailed_links.append(link)

#we create a list of all links to fatalaty data, csv_links
csv_fatalities_links = []        
for link in all_links:
    if((link[-6:-1] == 'csv.g') and link.startswith('StormEvents_fatalities')):
        csv_fatalities_links.append(link)
        
#we create a list of all links to location data, csv_links
csv_location_links = []        
for link in all_links:
    if((link[-6:-1] == 'csv.g') and link.startswith('StormEvents_locations')):
        csv_location_links.append(link)        

In [8]:
detailed_df = pd.DataFrame()
for i in csv_detailed_links:
    detailed_df = pd.concat([detailed_df,pd.read_csv(https + i, index_col = 'EVENT_ID')])

#we'll run these later    
# fatalities_df = pd.DataFrame()
# for i in csv_fatalities_links:
#     fatalities_df = pd.concat([fatalities_df,pd.read_csv(https + i)])
    
# location_df = pd.DataFrame()
# for i in csv_location_links:
#     location_df = pd.concat([location_df,pd.read_csv(https + i)])

In [9]:
detailed_df.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,STATE,STATE_FIPS,YEAR,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
EVENT_ID,,,,,,,,,,,,,,,,,,,,,
10096222,195004,28,1445,195004,28,1445,NaN,OKLAHOMA,40.0,1950,...,0.0,NaN,NaN,35.12,-99.20,35.17,-99.20,NaN,NaN,PUB
10120412,195004,29,1530,195004,29,1530,NaN,TEXAS,48.0,1950,...,0.0,NaN,NaN,31.90,-98.60,31.73,-98.60,NaN,NaN,PUB
10104927,195007,5,1800,195007,5,1800,NaN,PENNSYLVANIA,42.0,1950,...,0.0,NaN,NaN,40.58,-75.70,40.65,-75.47,NaN,NaN,PUB
10104928,195007,5,1830,195007,5,1830,NaN,PENNSYLVANIA,42.0,1950,...,0.0,NaN,NaN,40.60,-76.75,NaN,NaN,NaN,NaN,PUB
10104929,195007,24,1440,195007,24,1440,NaN,PENNSYLVANIA,42.0,1950,...,0.0,NaN,NaN,41.63,-79.68,NaN,NaN,NaN,NaN,PUB


In [16]:
#CHECKS TO MAKE SURE THERE ARE NO DUPLICATES. IT'S ALL GOOD!!!!

# #detailed_df.sort_index()

# idx = pd.Index(detailed_df)
# idx.duplicated()

# from collections import Counter

# words = idx.duplicated()

# print(Counter(words).keys()) # equals to list(set(words))
# print(Counter(words).values()) # counts the elements' frequency

dict_keys([False])
dict_values([1520416])


In [ ]:
detailed_df.columns

In [ ]:
detailed_df.info()

In [18]:
#removing any column that has more than 60% of data as NA
test = detailed_df.isnull().sum() < .6*detailed_df.shape[0]

In [19]:
test[0]

True

In [20]:
columns = [test.index[i] for i in range(len(test)) if test[i]]
columns

['BEGIN_YEARMONTH',
 'BEGIN_DAY',
 'BEGIN_TIME',
 'END_YEARMONTH',
 'END_DAY',
 'END_TIME',
 'EPISODE_ID',
 'STATE',
 'STATE_FIPS',
 'YEAR',
 'MONTH_NAME',
 'EVENT_TYPE',
 'CZ_TYPE',
 'CZ_FIPS',
 'CZ_NAME',
 'WFO',
 'BEGIN_DATE_TIME',
 'CZ_TIMEZONE',
 'END_DATE_TIME',
 'INJURIES_DIRECT',
 'INJURIES_INDIRECT',
 'DEATHS_DIRECT',
 'DEATHS_INDIRECT',
 'DAMAGE_PROPERTY',
 'DAMAGE_CROPS',
 'SOURCE',
 'MAGNITUDE',
 'BEGIN_RANGE',
 'BEGIN_LOCATION',
 'END_RANGE',
 'END_LOCATION',
 'BEGIN_LAT',
 'BEGIN_LON',
 'END_LAT',
 'END_LON',
 'EPISODE_NARRATIVE',
 'EVENT_NARRATIVE',
 'DATA_SOURCE']

In [21]:
detailed_df_remove_na = detailed_df[columns]

In [22]:
detailed_df_remove_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520416 entries, 10096222 to 743878
Data columns (total 38 columns):
BEGIN_YEARMONTH      1520416 non-null int64
BEGIN_DAY            1520416 non-null int64
BEGIN_TIME           1520416 non-null int64
END_YEARMONTH        1520416 non-null int64
END_DAY              1520416 non-null int64
END_TIME             1520416 non-null int64
EPISODE_ID           1288177 non-null float64
STATE                1520415 non-null object
STATE_FIPS           1520415 non-null float64
YEAR                 1520416 non-null int64
MONTH_NAME           1520416 non-null object
EVENT_TYPE           1520416 non-null object
CZ_TYPE              1520416 non-null object
CZ_FIPS              1520416 non-null int64
CZ_NAME              1518859 non-null object
WFO                  1394843 non-null object
BEGIN_DATE_TIME      1520416 non-null object
CZ_TIMEZONE          1520416 non-null object
END_DATE_TIME        1520416 non-null object
INJURIES_DIRECT      1520416 non

In [23]:
detailed_df_remove_na.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,STATE,STATE_FIPS,YEAR,...,BEGIN_LOCATION,END_RANGE,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
EVENT_ID,,,,,,,,,,,,,,,,,,,,,
10096222,195004,28,1445,195004,28,1445,NaN,OKLAHOMA,40.0,1950,...,NaN,0.0,NaN,35.12,-99.20,35.17,-99.20,NaN,NaN,PUB
10120412,195004,29,1530,195004,29,1530,NaN,TEXAS,48.0,1950,...,NaN,0.0,NaN,31.90,-98.60,31.73,-98.60,NaN,NaN,PUB
10104927,195007,5,1800,195007,5,1800,NaN,PENNSYLVANIA,42.0,1950,...,NaN,0.0,NaN,40.58,-75.70,40.65,-75.47,NaN,NaN,PUB
10104928,195007,5,1830,195007,5,1830,NaN,PENNSYLVANIA,42.0,1950,...,NaN,0.0,NaN,40.60,-76.75,NaN,NaN,NaN,NaN,PUB
10104929,195007,24,1440,195007,24,1440,NaN,PENNSYLVANIA,42.0,1950,...,NaN,0.0,NaN,41.63,-79.68,NaN,NaN,NaN,NaN,PUB


In [24]:
#prints the number of unique values in each column
[print(column, len(detailed_df_remove_na[column].unique())) for column in detailed_df_remove_na.columns]

BEGIN_YEARMONTH 820
BEGIN_DAY 31
BEGIN_TIME 1440
END_YEARMONTH 820
END_DAY 31
END_TIME 1440
EPISODE_ID 321702
STATE 69
STATE_FIPS 69
YEAR 69
MONTH_NAME 12
EVENT_TYPE 74
CZ_TYPE 4
CZ_FIPS 591
CZ_NAME 5246
WFO 542
BEGIN_DATE_TIME 819667
CZ_TIMEZONE 26
END_DATE_TIME 809656
INJURIES_DIRECT 200
INJURIES_INDIRECT 58
DEATHS_DIRECT 54
DEATHS_INDIRECT 14
DAMAGE_PROPERTY 2746
DAMAGE_CROPS 1154
SOURCE 74
MAGNITUDE 435
BEGIN_RANGE 155
BEGIN_LOCATION 60002
END_RANGE 150
END_LOCATION 48926
BEGIN_LAT 102321
BEGIN_LON 128279
END_LAT 109351
END_LON 138792
EPISODE_NARRATIVE 188566
EVENT_NARRATIVE 600457
DATA_SOURCE 5


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [25]:
#what I'm thinking now is that there is so much data already... 1.5million+ rows, it might be okay to just remove rows with na
detailed_df_remove_na.shape

(1520416, 38)

In [26]:
det_df = detailed_df_remove_na.dropna()

In [27]:
det_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192145 entries, 5698838 to 743878
Data columns (total 38 columns):
BEGIN_YEARMONTH      192145 non-null int64
BEGIN_DAY            192145 non-null int64
BEGIN_TIME           192145 non-null int64
END_YEARMONTH        192145 non-null int64
END_DAY              192145 non-null int64
END_TIME             192145 non-null int64
EPISODE_ID           192145 non-null float64
STATE                192145 non-null object
STATE_FIPS           192145 non-null float64
YEAR                 192145 non-null int64
MONTH_NAME           192145 non-null object
EVENT_TYPE           192145 non-null object
CZ_TYPE              192145 non-null object
CZ_FIPS              192145 non-null int64
CZ_NAME              192145 non-null object
WFO                  192145 non-null object
BEGIN_DATE_TIME      192145 non-null object
CZ_TIMEZONE          192145 non-null object
END_DATE_TIME        192145 non-null object
INJURIES_DIRECT      192145 non-null int64
INJURIES_I

Let's see which columns need cleaning...
Referencing: https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/Storm-Data-Export-Format.docx

EPISODE_ID: float --> object/int 
CZ_FIPS: float --> object

BEGIN_DATE_TIME: object --> datetime
CZ_TIMEZONE
END_DATE_TIME: object --> datetime

**DAMAGE_PROPERTY: object --> int/float
DAMAGE_CROPS: object --> int/float

In [ ]:
#created function to clean strings ending in "K" and "M"
#input dataframe and column name
def clean_nums(df, column):
    for i in range(len(df[column])):
        if type(df.loc[i, column]) == float:
            pass
        elif df.loc[i, column] == 'K':
            df.set_value(i, column, 1000)
        elif df.loc[i,column].endswith('K'):
            df.loc[i, column] = df.loc[i, column].replace("K", "")
            df.set_value(i, column, (float(df.loc[i, column]) * 1000))
        elif df.loc[i,column].endswith('M'):
            df.loc[i, column] = df.loc[i, column].replace("M", "")
            df.set_value(i, column, (float(df.loc[i, column]) * 1000000))
        else:
            df.set_value(i, column, float(df.loc[i,column]))
    df[column] = df[column].astype(float)

# BELOW is the testcase
#--------------------------------------------
# test_case = det_df.head(2000)
# test_case = test_case.reset_index(drop=True)
# print(test_case['DAMAGE_CROPS'])
# clean_nums(test_case, 'DAMAGE_CROPS')
# print(test_case['DAMAGE_CROPS'])

In [28]:
df_new = det_df[['DAMAGE_PROPERTY']]
part1 = df_new[df_new['DAMAGE_PROPERTY'].map(lambda x: x[-1] == 'K')]['DAMAGE_PROPERTY'].map(lambda x: float(x[:-1]) * 1000)
part2 = df_new[df_new['DAMAGE_PROPERTY'].map(lambda x: x[-1] == 'M')]['DAMAGE_PROPERTY'].map(lambda x: float(x[:-1]) * 1000000)
part3 = df_new[df_new['DAMAGE_PROPERTY'].map(lambda x: x[-1] == 'B')]['DAMAGE_PROPERTY'].map(lambda x: float(x[:-1]) * 1000000000)
part4 = df_new[df_new['DAMAGE_PROPERTY'].map(lambda x: x[-1] != 'K' and x[-1] != 'M' and x[-1] != 'B')]['DAMAGE_PROPERTY']

In [30]:
det_df.sort_index()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,STATE,STATE_FIPS,YEAR,...,BEGIN_LOCATION,END_RANGE,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
EVENT_ID,,,,,,,,,,,,,,,,,,,,,
15,200610,15,1117,200610,15,1119,10.0,TEXAS,48.0,2006,...,PLAINVIEW,0.0,PLAINVIEW,34.20,-101.70,34.20,-101.70,A round of strong to severe thunderstorms and ...,A strong thunderstorm produced a series of wet...,CSV
47,200610,16,1855,200610,16,1855,15.0,NEBRASKA,31.0,2006,...,BENKELMAN JONES ARPT,1.0,BENKELMAN JONES ARPT,40.05,-101.54,40.05,-101.54,Storms intensified along the Nebraska-Kansas b...,Dime size hail covered the ground.,CSV
77,200610,27,2045,200610,27,2045,24.0,FLORIDA,12.0,2006,...,ELLAVILLE,1.0,ELLAVILLE,30.38,-83.17,30.38,-83.17,A squall line moved out of the Florida Big Ben...,Several trees down in the Suwannee River State...,CSV
78,200610,27,2130,200610,27,2130,24.0,FLORIDA,12.0,2006,...,TRENTON,1.0,TRENTON,29.61,-82.82,29.61,-82.82,A squall line moved out of the Florida Big Ben...,"Several trees down in the Trenton, Fl. area. T...",CSV
79,200610,27,2130,200610,27,2130,24.0,FLORIDA,12.0,2006,...,BRANFORD,2.0,BRANFORD,29.96,-82.89,29.96,-82.89,A squall line moved out of the Florida Big Ben...,Power Lines blown down near U.S. Highway 27 an...,CSV
95,200610,5,1900,200610,5,1905,32.0,TENNESSEE,47.0,2006,...,CLINTON,0.0,CLINTON,36.12,-84.13,36.12,-84.13,Scattered Severe Thunderstorms covered the nor...,A few trees were reported down near Clinton by...,CSV
96,200610,5,1905,200610,5,1910,32.0,TENNESSEE,47.0,2006,...,AMHERST,0.0,AMHERST,35.95,-84.03,35.95,-84.03,Scattered Severe Thunderstorms covered the nor...,Trees down across northern Knox County.,CSV
97,200610,5,1910,200610,5,1915,32.0,TENNESSEE,47.0,2006,...,AMHERST,0.0,AMHERST,35.95,-84.03,35.95,-84.03,Scattered Severe Thunderstorms covered the nor...,Trees down in northeast Knox County.,CSV
98,200610,5,1940,200610,5,1945,32.0,TENNESSEE,47.0,2006,...,MASCOT,0.0,MASCOT,36.07,-83.75,36.07,-83.75,Scattered Severe Thunderstorms covered the nor...,Trees down on Spence Place in Southeast Knox C...,CSV


In [ ]:
det_df.head()

In [29]:
part_all = pd.concat([part1, part2, part3, part4])
det_df_clean = det_df.copy()
# det_df.sort_index(inplace=True)
# part_all.sort_index(inplace=True)
det_df_clean['DAMAGE_PROPERTY_CLEAN'] = part_all
# det_df_clean.append(part_all)

In [31]:
det_df_clean.sort_index()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,STATE,STATE_FIPS,YEAR,...,END_RANGE,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE,DAMAGE_PROPERTY_CLEAN
EVENT_ID,,,,,,,,,,,,,,,,,,,,,
15,200610,15,1117,200610,15,1119,10.0,TEXAS,48.0,2006,...,0.0,PLAINVIEW,34.20,-101.70,34.20,-101.70,A round of strong to severe thunderstorms and ...,A strong thunderstorm produced a series of wet...,CSV,0
47,200610,16,1855,200610,16,1855,15.0,NEBRASKA,31.0,2006,...,1.0,BENKELMAN JONES ARPT,40.05,-101.54,40.05,-101.54,Storms intensified along the Nebraska-Kansas b...,Dime size hail covered the ground.,CSV,0
77,200610,27,2045,200610,27,2045,24.0,FLORIDA,12.0,2006,...,1.0,ELLAVILLE,30.38,-83.17,30.38,-83.17,A squall line moved out of the Florida Big Ben...,Several trees down in the Suwannee River State...,CSV,5000
78,200610,27,2130,200610,27,2130,24.0,FLORIDA,12.0,2006,...,1.0,TRENTON,29.61,-82.82,29.61,-82.82,A squall line moved out of the Florida Big Ben...,"Several trees down in the Trenton, Fl. area. T...",CSV,5000
79,200610,27,2130,200610,27,2130,24.0,FLORIDA,12.0,2006,...,2.0,BRANFORD,29.96,-82.89,29.96,-82.89,A squall line moved out of the Florida Big Ben...,Power Lines blown down near U.S. Highway 27 an...,CSV,5000
95,200610,5,1900,200610,5,1905,32.0,TENNESSEE,47.0,2006,...,0.0,CLINTON,36.12,-84.13,36.12,-84.13,Scattered Severe Thunderstorms covered the nor...,A few trees were reported down near Clinton by...,CSV,0
96,200610,5,1905,200610,5,1910,32.0,TENNESSEE,47.0,2006,...,0.0,AMHERST,35.95,-84.03,35.95,-84.03,Scattered Severe Thunderstorms covered the nor...,Trees down across northern Knox County.,CSV,0
97,200610,5,1910,200610,5,1915,32.0,TENNESSEE,47.0,2006,...,0.0,AMHERST,35.95,-84.03,35.95,-84.03,Scattered Severe Thunderstorms covered the nor...,Trees down in northeast Knox County.,CSV,0
98,200610,5,1940,200610,5,1945,32.0,TENNESSEE,47.0,2006,...,0.0,MASCOT,36.07,-83.75,36.07,-83.75,Scattered Severe Thunderstorms covered the nor...,Trees down on Spence Place in Southeast Knox C...,CSV,0


In [ ]:
# det_def.head()
print(len(det_df))
print(len(part1)+len(part2)+len(part3)+len(part4))


In [ ]:
det_df_clean['DAMAGE_PROPERTY_CLEAN'].head()

In [ ]:
# det_df_clean = det_df.reset_index(drop=True)
# clean_nums(det_df_clean, 'DAMAGE_PROPERTY')
# clean_nums(det_df_clean, 'DAMAGE_CROPS')

#Think this took an hour to run....

In [ ]:
det_df_clean.to_csv("US_Storm_Events_noNA_clean.csv")

In [ ]:
#det_df_clean['DAMAGE_PROPERTY'].unique()
#det_df['DAMAGE_CROPS'].unique()
#len(det_df['DAMAGE_PROPERTY'].endswith('K'))

len([row for row in det_df['DAMAGE_PROPERTY'] if row.endswith('K')])

# 